In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
py_file_location = "/content/drive/MyDrive/GNN_Seg/"
sys.path.append(os.path.abspath(py_file_location))

In [ ]:
pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 4.7 MB/s 
     |████████████████████████████████| 280 kB 76.8 MB/s 
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [ ]:
pip install torchviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4149 sha256=8d3d29e66e03435cacf778fd99c17054ae270e78ab8806171b00dd84b9c92a49
  Stored in directory: /root/.cache/pip/wheels/04/38/f5/dc4f85c3909051823df49901e72015d2d750bd26b086480ec2
Successfully built torchviz


In [ ]:
import argparse
from numpy import r_, around
from torch.utils.data import Subset
import torch
from utils.training_helpers import *
from utils.hyperparam_helpers import populate_hardcoded_hyperparameters, generate_random_hyperparameters
from data_processing.data_loader import ImageGraphDataset
from model.gnn_model import GNN
from model.networks import init_graph_net

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from utils.hyperparam_helpers import EvalParamSet,DEFAULT_BACKGROUND_NODE_LOGITS,populate_hardcoded_hyperparameters
import Filepaths

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
import torch
from torch import nn
%pip install -U git+https://github.com/szagoruyko/pytorchviz.git@master
from torchviz import make_dot, make_dot_from_trace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/szagoruyko/pytorchviz.git (to revision master) to /tmp/pip-req-build-n1rytmgy
  Running command git clone -q https://github.com/szagoruyko/pytorchviz.git /tmp/pip-req-build-n1rytmgy


In [ ]:
import torch
from torchsummary import summary
from torchvision import models
# Model class must be defined somewhere
att_heads = [4, 5, 5, 5, 3, 5, 5, 3]
residuals = [False, False, False, False, True, False, False, False]
gnn_hp = EvalParamSet(in_feats=20,out_classes=4,layer_sizes=[256]*8,gat_heads=att_heads,gat_residuals=residuals)
net = init_graph_net('GAT',gnn_hp)

net.load_state_dict(torch.load('/content/drive/MyDrive/GAT_gnn_f1.pt',map_location=device))
print(net)

GAT(
  (layers): ModuleList(
    (0): GATConv(
      (fc): Linear(in_features=20, out_features=1024, bias=False)
      (feat_drop): Dropout(p=0, inplace=False)
      (attn_drop): Dropout(p=0, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
    (1): GATConv(
      (fc): Linear(in_features=1024, out_features=1280, bias=False)
      (feat_drop): Dropout(p=0, inplace=False)
      (attn_drop): Dropout(p=0, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
    (2): GATConv(
      (fc): Linear(in_features=1280, out_features=1280, bias=False)
      (feat_drop): Dropout(p=0, inplace=False)
      (attn_drop): Dropout(p=0, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
    (3): GATConv(
      (fc): Linear(in_features=1280, out_features=1280, bias=False)
      (feat_drop): Dropout(p=0, inplace=False)
      (attn_drop): Dropout(p=0, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
    (4): GATConv(
      (fc):

In [ ]:
from prettytable import PrettyTable
def count_parameters(net):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in net.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [ ]:
count_parameters(net)

+------------------------+------------+
|        Modules         | Parameters |
+------------------------+------------+
|    layers.0.attn_l     |    1024    |
|    layers.0.attn_r     |    1024    |
|     layers.0.bias      |    1024    |
|   layers.0.fc.weight   |   20480    |
|    layers.1.attn_l     |    1280    |
|    layers.1.attn_r     |    1280    |
|     layers.1.bias      |    1280    |
|   layers.1.fc.weight   |  1310720   |
|    layers.2.attn_l     |    1280    |
|    layers.2.attn_r     |    1280    |
|     layers.2.bias      |    1280    |
|   layers.2.fc.weight   |  1638400   |
|    layers.3.attn_l     |    1280    |
|    layers.3.attn_r     |    1280    |
|     layers.3.bias      |    1280    |
|   layers.3.fc.weight   |  1638400   |
|    layers.4.attn_l     |    768     |
|    layers.4.attn_r     |    768     |
|     layers.4.bias      |    768     |
|   layers.4.fc.weight   |   983040   |
| layers.4.res_fc.weight |   983040   |
|    layers.5.attn_l     |    1280    |


10208524

In [ ]:
from dgl.nn.pytorch import GATConv

